<div style="background-color: black;">
<hr style="border: 3px solid skyblue;">
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman; color: skyblue; text-transform: uppercase;">
    TIME SERIES DATA PROCESSING
    <br>
    AVAILABILITY FACTOR | BOUNDARY SECTOR DEMAND | SCALLED INFLOWS | RESERVOIR LEVELS | LOAD REAL TIME | LOAD DAY AHEAD
</div>
<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color: skyblue">
Fulfilling 15 minutes Not Available Time Steps Notebook
</div>
<div style="text-align: justify; margin-left: 0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color: skyblue">
        Each part of the following script was used for the pacially processed time series i.e. Availability Factors, Scalled Inflows, Current Load and Load Dayahead data, to cumpliment all those data which is missing of 15 and 30 minute time steps.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
    <hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    1. Notebook Set Up
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Importing needed libraries
</div>

In [1]:
import os
import csv
from datetime import datetime
import pytz
from pytz import timezone, utc
import shutil
from functools import lru_cache
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from datetime import timedelta

<div style="background-color: black;">
<hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    2. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [2]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="background-color: black;">
<hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    3. Usefull Variable Definition
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: Justify; margin-left: 2.0em; font-weight: unbold; font-size: 13px; font-family: TimesNewRoman; color:skyblue">
Indicate the year of all data is referring to in the variable data_year.
<br>
The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [3]:
# Year to which data refers to:
data_years_list = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [4]:
time_steps_list = ['1h', '30min', '15min']

In [5]:
# Countries to which data refers to:
countries_list = [
    'AL', 'BA', 'BG', 'CY', 'DE', 'EE', 'ES', 'FR', 'HR', 'IE', 'LT', 'LV', 'ME', 'MT', 'NO', 'PT', 'RS', 'SI', 'UK',
    'AT', 'BE', 'CH', 'CZ', 'DK', 'EL', 'FI', 'GR', 'HU', 'IT', 'LU', 'MD', 'MK', 'NL', 'PL', 'RO', 'SE', 'SK'
]

In [6]:
# Additional string to be appended
availability_factors_folder_path = os.path.join(dispaSET_unleash_folder_path, "Database", "AvailabilityFactors")
load_dayahead_folder_path = os.path.join(dispaSET_unleash_folder_path, "Database", "Load_DayAhead")
load_realtime_folder_path = os.path.join(dispaSET_unleash_folder_path, "Database", "Load_RealTime")
scalled_inflows_folder_path = os.path.join(dispaSET_unleash_folder_path, "Database", "HydroData", "ScaledInflows")
reservoir_level_folder_path = os.path.join(dispaSET_unleash_folder_path, "Database", "HydroData", "ReservoirLevel")
boundary_sector_demand_folder_path = os.path.join(dispaSET_unleash_folder_path, "Database", "BoudarySector", "BoudarySectorDemand")

print("availability_factors_folder_path:", availability_factors_folder_path)
print("load_dayahead_folder_path:", load_dayahead_folder_path)
print("load_realtime_folder_path:", load_realtime_folder_path)
print("scalled_inflows_folder_path:", scalled_inflows_folder_path)
print("reservoir_level_folder_path:", reservoir_level_folder_path)
print("boundary_sector_demand_folder_path:", boundary_sector_demand_folder_path)

availability_factors_folder_path: /home/ray/Dispa-SET_Unleash/Database/AvailabilityFactors
load_dayahead_folder_path: /home/ray/Dispa-SET_Unleash/Database/Load_DayAhead
load_realtime_folder_path: /home/ray/Dispa-SET_Unleash/Database/Load_RealTime
scalled_inflows_folder_path: /home/ray/Dispa-SET_Unleash/Database/HydroData/ScaledInflows
reservoir_level_folder_path: /home/ray/Dispa-SET_Unleash/Database/HydroData/ReservoirLevel
boundary_sector_demand_folder_path: /home/ray/Dispa-SET_Unleash/Database/BoudarySector/BoudarySectorDemand


<div style="background-color: black;">
<hr style="border: 2px solid skyblue;">
<div style="text-align: justify; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman; color:skyblue">
    4. Missed Time Series Data Fulfilling
</div>
<div style="text-align: justify; margin-left: 0.0em; font-weight: unbold; font-size: 14px; font-family: TimesNewRoman; color:skyblue">
Because of the data from the original sources, there are not some time series whit the finest granularity; e.g. 15 and 30 minutes.
<br>
This missed data is completed interpolating the data from the closer time step series in terms of resolution for all the years for the respective countries.
</div>

In [11]:
# Paths
all_data_paths = [
    availability_factors_folder_path,
    load_dayahead_folder_path,
    load_realtime_folder_path,
    scalled_inflows_folder_path,
    reservoir_level_folder_path,
    boundary_sector_demand_folder_path,
]

# Time step mapping
time_deltas = {'1h': timedelta(hours=1), '30min': timedelta(minutes=30), '15min': timedelta(minutes=15)}

def interpolate_and_save(df, target_step, output_path):
    df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
    df.set_index(df.columns[0], inplace=True)

    new_index = pd.date_range(start=df.index[0], end=df.index[-1], freq=target_step)
    df_resampled = df.reindex(new_index).interpolate(method='time')
    df_resampled.reset_index(inplace=True)
    df_resampled.rename(columns={df_resampled.columns[0]: ''}, inplace=True)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df_resampled.to_csv(output_path, index=False)

def process_directory(base_path):
    for country in countries_list:
        country_path = os.path.join(base_path, country)
        if not os.path.isdir(country_path):
            continue

        present_time_steps = [ts for ts in time_steps_list if os.path.isdir(os.path.join(country_path, ts))]

        for target_ts in time_steps_list:
            for year in data_years_list:
                output_file = os.path.join(country_path, target_ts, f'{year}.csv')
                if os.path.isfile(output_file):
                    continue  # Skip if already exists

                # Select source time step based on priority
                source_ts = None
                if target_ts == '15min':
                    if '30min' in present_time_steps:
                        source_ts = '30min'
                    elif '1h' in present_time_steps:
                        source_ts = '1h'
                elif target_ts == '30min' and '1h' in present_time_steps:
                    source_ts = '1h'

                if not source_ts:
                    continue

                source_file = os.path.join(country_path, source_ts, f'{year}.csv')
                if not os.path.isfile(source_file):
                    continue

                df = pd.read_csv(source_file)
                interpolate_and_save(df, target_ts, output_file)

# Run for all base directories
for folder in all_data_paths:
    process_directory(folder)


/home/ray/anaconda3/lib/python3.12/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/home/ray/anaconda3/lib/python3.12/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/home/ray/anaconda3/lib/python3.12/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
